# Multi-Agent HR System - LangGraph & LangChain 1.0

## Overview
Modern multi-agent patterns using **LangGraph** and **LangChain 1.0**:


## Setup and Installation

In [1]:
# Install required packages
!pip install -q -U langchain-openai langgraph langchain-core langchain python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 13.4 MB/s eta 0:00:00


In [ ]:
# Import libraries
import os
from dotenv import load_dotenv
from typing import TypedDict, Annotated, Literal
import operator
from datetime import datetime
import json

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage, ToolMessage
from langchain_core.tools import tool

# LangGraph imports
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command, interrupt

# LangChain 1.0 agent creation (NEW IMPORT)
from langchain.agents import create_agent

# Load environment variables
load_dotenv()

# Verify OpenAI API key
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY in your environment")

print("✓ Setup complete with LangGraph + LangChain 1.0!")

## Sample HR Data

In [ ]:
# Sample candidate data
SAMPLE_RESUME = """
PRIYA SHARMA
Senior Software Engineer
priya.sharma@email.com | +91-98765-43210 | Bengaluru, Karnataka

SUMMARY
6+ years in full-stack development, specializing in Python, React, and AWS.
Led teams, built microservices, mentored developers.

EXPERIENCE
Senior Software Engineer | InfoTech Solutions, Bengaluru | 2021-Present
- Led 4-member team building microservices architecture
- Reduced API latency by 45% through optimization
- Implemented CI/CD pipeline (Jenkins, Docker, K8s)
- Tech: Python, FastAPI, React, PostgreSQL, AWS

Software Engineer | Digital Innovations, Pune | 2018-2020
- Built web apps serving 500K+ users
- Integrated payment gateways (Razorpay, PayU)
- Tech: Python, Django, MySQL, AWS

EDUCATION
B.Tech CSE | BITS Pilani | 2016 | CGPA: 8.5/10

SKILLS
Python, JavaScript, FastAPI, React, AWS, Docker, Kubernetes, PostgreSQL

CERTIFICATIONS
AWS Solutions Architect - Associate (2022)
"""

JOB_DESCRIPTION = """
Senior Backend Engineer - TechCorp India, Bengaluru

Requirements:
- 5+ years backend development
- Python/FastAPI expertise
- AWS/cloud experience
- Microservices architecture
- Team leadership skills

Offer: ₹28-35 LPA + equity
"""

# Mock HR Database
HR_DATABASE = {
    "candidates": {
        "CAN001": {
            "name": "Priya Sharma",
            "email": "priya.sharma@email.com",
            "phone": "+91-98765-43210",
            "position": "Senior Backend Engineer",
            "status": "screening",
            "resume": SAMPLE_RESUME
        },
        "CAN002": {
            "name": "Arjun Mehta",
            "email": "arjun.mehta@email.com",
            "phone": "+91-98123-45678",
            "position": "Senior Backend Engineer",
            "status": "interview_scheduled"
        }
    },
    "employees": {
        "EMP001": {"name": "Rahul Verma", "role": "Engineering Manager", "email": "rahul.verma@techcorp.in"},
        "EMP002": {"name": "Anjali Patel", "role": "HR Manager", "email": "anjali.patel@techcorp.in"},
        "EMP003": {"name": "Vikram Singh", "role": "Tech Lead", "email": "vikram.singh@techcorp.in"}
    },
    "interview_slots": [
        {"date": "2025-10-15", "time": "10:00", "interviewer": "Rahul Verma", "available": True},
        {"date": "2025-10-15", "time": "14:00", "interviewer": "Vikram Singh", "available": True},
        {"date": "2025-10-16", "time": "11:00", "interviewer": "Anjali Patel", "available": True}
    ]
}

print(f"✓ HR Database loaded: {len(HR_DATABASE['candidates'])} candidates, {len(HR_DATABASE['employees'])} employees")

## Define HR Tools (Used across all labs)

In [ ]:
# Define HR Tools

@tool
def get_candidate_info(candidate_id: str) -> str:
    """
    Get candidate information from HR database.

    Args:
        candidate_id: Candidate ID like CAN001

    Returns:
        JSON string with candidate details
    """
    candidate = HR_DATABASE["candidates"].get(candidate_id)
    if candidate:
        return json.dumps(candidate, indent=2)
    return f"Candidate {candidate_id} not found"

@tool
def search_candidates(position: str) -> str:
    """
    Search for candidates by position.

    Args:
        position: Job position to search for

    Returns:
        List of matching candidates
    """
    results = []
    for cid, cand in HR_DATABASE["candidates"].items():
        if position.lower() in cand["position"].lower():
            results.append({"id": cid, "name": cand["name"], "status": cand["status"]})
    return json.dumps(results, indent=2)

@tool
def check_interview_availability(interviewer_name: str = None) -> str:
    """
    Check available interview slots.

    Args:
        interviewer_name: Optional filter by interviewer

    Returns:
        Available interview slots
    """
    slots = HR_DATABASE["interview_slots"]

    if interviewer_name:
        slots = [s for s in slots if interviewer_name.lower() in s["interviewer"].lower()]

    available = [s for s in slots if s["available"]]
    return json.dumps(available, indent=2)

@tool
def update_candidate_status(candidate_id: str, new_status: str) -> str:
    """
    Update candidate status in ATS.

    Args:
        candidate_id: Candidate ID
        new_status: New status (screening, interview_scheduled, offer, rejected)

    Returns:
        Confirmation message
    """
    if candidate_id in HR_DATABASE["candidates"]:
        old_status = HR_DATABASE["candidates"][candidate_id]["status"]
        HR_DATABASE["candidates"][candidate_id]["status"] = new_status
        return f"✓ Updated {HR_DATABASE['candidates'][candidate_id]['name']}: {old_status} → {new_status}"
    return f"❌ Candidate {candidate_id} not found"

@tool
def send_email_notification(to_email: str, subject: str, message: str) -> str:
    """
    Send email notification.

    Args:
        to_email: Recipient email
        subject: Email subject
        message: Email body

    Returns:
        Confirmation
    """
    return f"✓ Email sent to {to_email}\nSubject: {subject}\n[SIMULATED]"

# Collect all tools
hr_tools = [
    get_candidate_info,
    search_candidates,
    check_interview_availability,
    update_candidate_status,
    send_email_notification
]

print("✓ HR Tools defined:")
for t in hr_tools:
    print(f"  - {t.name}")

---

# LAB 6: SUPERVISOR PATTERN 🆕

**Pattern**: Based on LangChain JavaScript supervisor documentation  
**Reference**: https://docs.langchain.com/oss/javascript/langchain/supervisor

## Architecture (3 Layers)

```
┌─────────────────────────────────────────────────┐
│  TOP LAYER: Supervisor Agent                   │
│  - Routes requests to appropriate specialist   │
│  - Synthesizes responses                        │
│  - Makes high-level decisions                   │
└─────────────────────────────────────────────────┘
                     │
        ┌────────────┴────────────┐
        ▼                         ▼
┌──────────────────┐    ┌──────────────────┐
│ MIDDLE LAYER     │    │ MIDDLE LAYER     │
│ Recruitment      │    │ Interview        │
│ Specialist       │    │ Coordinator      │
│ (Sub-agent)      │    │ (Sub-agent)      │
└──────────────────┘    └──────────────────┘
        │                         │
   ┌────┴─────┐            ┌─────┴────┐
   ▼          ▼            ▼          ▼
┌─────┐  ┌─────┐      ┌─────┐  ┌─────┐
│Tool │  │Tool │      │Tool │  │Tool │
│ 1   │  │ 2   │      │ 3   │  │ 4   │
└─────┘  └─────┘      └─────┘  └─────┘
BOTTOM LAYER: Individual API Tools
```

## Key Benefits
1. **Separation of Concerns**: Each layer has focused responsibility
2. **Scalability**: Add new specialists without affecting existing ones
3. **Maintainability**: Test and iterate on each layer independently
4. **Natural Language Interface**: Supervisor handles high-level requests

## Step 1: Create Specialized Sub-Agents (Middle Layer)

In [ ]:
# ============================================================================
# MIDDLE LAYER: Specialized Sub-Agents
# ============================================================================
# Using LangChain 1.0 create_agent with system_prompt parameter

# Sub-Agent 1: Recruitment Specialist
# Handles: candidate search, profile review, status updates
recruitment_specialist_agent = create_agent(
    model="openai:gpt-4o",
    tools=[get_candidate_info, search_candidates, update_candidate_status],
    system_prompt="""You are a Recruitment Specialist at TechCorp India.

Your expertise:
- Searching for candidates in the database
- Reviewing candidate profiles and qualifications
- Updating candidate status in the ATS
- Assessing candidate fit for positions

You ONLY handle recruitment-related tasks. You do NOT schedule interviews.
Provide detailed, professional assessments using the available tools.

Return natural language responses about your findings."""
)

# Sub-Agent 2: Interview Coordinator
# Handles: scheduling, availability checks, notifications
interview_coordinator_agent = create_agent(
    model="openai:gpt-4o",
    tools=[check_interview_availability, send_email_notification, update_candidate_status],
    system_prompt="""You are an Interview Coordinator at TechCorp India.

Your expertise:
- Checking interviewer availability
- Scheduling interview sessions
- Sending interview confirmations and reminders
- Managing interview logistics

You ONLY handle interview scheduling and coordination.
You do NOT search for candidates or assess qualifications.

Return natural language confirmations of scheduled interviews."""
)

print("✓ Sub-agents created using LangChain 1.0 create_agent:")
print("  1. Recruitment Specialist (search, review, assess)")
print("  2. Interview Coordinator (schedule, notify, coordinate)")

## Step 2: Wrap Sub-Agents as Tools for Supervisor

In [ ]:
# ============================================================================
# Wrap sub-agents as tools (KEY ARCHITECTURAL STEP)
# ============================================================================
# The supervisor sees "recruitment_specialist" and "interview_coordinator" as
# high-level tools, NOT the individual low-level API tools

@tool
def recruitment_specialist(request: str) -> str:
    """Handle recruitment tasks like candidate search, profile review, and assessments.

    Use this tool when you need to:
    - Search for candidates by position or criteria
    - Get detailed candidate information and profiles
    - Update candidate status after screening
    - Assess candidate qualifications and fit

    This specialist has access to the candidate database and ATS.

    Args:
        request: Natural language description of the recruitment task

    Returns:
        Natural language response from the recruitment specialist
    """
    result = recruitment_specialist_agent.invoke(
        {"messages": [HumanMessage(content=request)]}
    )

    # Extract the final AI response
    for msg in reversed(result['messages']):
        if isinstance(msg, AIMessage) and msg.content:
            return msg.content

    return "Recruitment specialist completed the task."

@tool
def interview_coordinator(request: str) -> str:
    """Handle interview scheduling and coordination tasks.

    Use this tool when you need to:
    - Check interviewer availability
    - Schedule interview sessions
    - Send interview confirmations to candidates and interviewers
    - Manage interview logistics and timing

    This coordinator has access to the calendar system and email.

    Args:
        request: Natural language description of the interview coordination task

    Returns:
        Natural language response from the interview coordinator
    """
    result = interview_coordinator_agent.invoke(
        {"messages": [HumanMessage(content=request)]}
    )

    # Extract the final AI response
    for msg in reversed(result['messages']):
        if isinstance(msg, AIMessage) and msg.content:
            return msg.content

    return "Interview coordinator completed the task."

# Supervisor tools (high-level only)
supervisor_tools = [
    recruitment_specialist,
    interview_coordinator
]

print("✓ Sub-agents wrapped as tools for supervisor")
print("\nSupervisor sees these HIGH-LEVEL capabilities:")
for t in supervisor_tools:
    print(f"  - {t.name}")
print("\nSupervisor does NOT see low-level tools like:")
print("  - get_candidate_info, search_candidates, etc.")
print("\nThis creates clean separation of concerns!")

## Step 3: Create the Supervisor Agent (Top Layer)

In [ ]:
# ============================================================================
# TOP LAYER: Supervisor Agent
# ============================================================================

supervisor_agent = create_agent(
    model="openai:gpt-4o",
    tools=supervisor_tools,
    system_prompt="""You are an HR Supervisor at TechCorp India.

Your role is to coordinate specialized HR agents and ensure smooth hiring operations.

You have access to two specialized agents:

1. **recruitment_specialist**: For candidate-related tasks
   - Searching for candidates
   - Reviewing profiles and qualifications
   - Assessing candidate fit
   - Updating candidate status after screening

2. **interview_coordinator**: For scheduling-related tasks
   - Checking interviewer availability
   - Scheduling interviews
   - Sending confirmations

**Your responsibilities:**
- Understand the user's request
- Route to the appropriate specialist (or both if needed)
- Synthesize responses from specialists into a clear summary
- Ensure all parts of complex requests are handled

**Decision making:**
- For candidate search/review → use recruitment_specialist
- For interview scheduling → use interview_coordinator
- For end-to-end hiring tasks → use BOTH specialists in sequence

Always provide a clear, professional summary after coordinating with specialists."""
)

print("✓ Supervisor agent created using LangChain 1.0")
print("\n🎯 The supervisor makes HIGH-LEVEL routing decisions:")
print("   - Does NOT choose between low-level tools")
print("   - Coordinates domain-level specialists")
print("   - Synthesizes multi-agent responses")

## Test Cases

In [ ]:
# ============================================================================
# Test Case 1: Single Domain Request (Recruitment Only)
# ============================================================================

print("\n" + "="*80)
print("🧪 TEST 1: Single Domain - Recruitment Only")
print("="*80)

request1 = "Find me all candidates applying for Senior Backend Engineer positions and give me a summary of their qualifications."

print(f"\nUser Request: {request1}")
print("\nExpected Flow: Supervisor → Recruitment Specialist only")
print("-" * 80)

result1 = supervisor_agent.invoke(
    {"messages": [HumanMessage(content=request1)]}
)

print("\n🤖 Supervisor's Response:")
for msg in result1['messages']:
    if isinstance(msg, AIMessage) and msg.content and not hasattr(msg, 'tool_calls'):
        print(msg.content)
        break

In [ ]:
# ============================================================================
# Test Case 2: Complex Multi-Domain Request
# ============================================================================

print("\n" + "="*80)
print("🧪 TEST 2: Multi-Domain - Full Hiring Workflow")
print("="*80)

request2 = """
I need help with candidate CAN001 (Priya Sharma):
1. First, get her profile and assess her qualifications
2. Then schedule an interview with Rahul Verma
3. Update her status accordingly
"""

print(f"\nUser Request: {request2}")
print("\nExpected Flow: Supervisor → Recruitment Specialist → Interview Coordinator")
print("The supervisor should coordinate BOTH specialists to complete the full workflow")
print("-" * 80)

result2 = supervisor_agent.invoke(
    {"messages": [HumanMessage(content=request2)]}
)

print("\n📊 Execution Trace:")
tool_calls_made = []
for msg in result2['messages']:
    if isinstance(msg, AIMessage) and hasattr(msg, 'tool_calls') and msg.tool_calls:
        for tc in msg.tool_calls:
            tool_calls_made.append(tc['name'])
            print(f"  ✓ Supervisor called: {tc['name']}")

print(f"\n📈 Total specialist calls: {len(tool_calls_made)}")

print("\n🤖 Supervisor's Final Response:")
for msg in reversed(result2['messages']):
    if isinstance(msg, AIMessage) and msg.content and not hasattr(msg, 'tool_calls'):
        print(msg.content)
        break

In [ ]:
# ============================================================================
# Test Case 3: Interview Coordination Only
# ============================================================================

print("\n" + "="*80)
print("🧪 TEST 3: Single Domain - Interview Coordination Only")
print("="*80)

request3 = "Check availability with Vikram Singh and send me the available interview slots."

print(f"\nUser Request: {request3}")
print("\nExpected Flow: Supervisor → Interview Coordinator only")
print("-" * 80)

result3 = supervisor_agent.invoke(
    {"messages": [HumanMessage(content=request3)]}
)

print("\n🤖 Supervisor's Response:")
for msg in result3['messages']:
    if isinstance(msg, AIMessage) and msg.content and not hasattr(msg, 'tool_calls'):
        print(msg.content)
        break

## Summary

### ✅ What We Demonstrated

1. **3-Layer Architecture**:
   - **Bottom Layer**: Raw tools (APIs) - `get_candidate_info`, `send_email_notification`, etc.
   - **Middle Layer**: Sub-agents that use tools and handle natural language
   - **Top Layer**: Supervisor that coordinates sub-agents

2. **Key Benefits**:
   - ✓ Supervisor makes high-level routing decisions (domain-level, not tool-level)
   - ✓ Each specialist has focused responsibility and clear boundaries
   - ✓ Easy to add new specialists without affecting existing ones
   - ✓ Sub-agents can be tested and improved independently
   - ✓ Natural language interface at all levels

3. **LangChain 1.0 Features Used**:
   - ✓ `create_agent` from `langchain.agents` (not deprecated `create_react_agent`)
   - ✓ `system_prompt` parameter (not `state_modifier`)
   - ✓ Model string format: `"openai:gpt-4o"`

### 🎯 When to Use Supervisor Pattern

Use this pattern when:
- Tasks require different types of expertise
- You have many tools that can be logically grouped by domain
- You want clear boundaries between different responsibilities
- You need to scale by adding new specialists
- You want independent testing and iteration of each specialist

### 📚 Reference

This pattern follows the LangChain JavaScript supervisor documentation:
https://docs.langchain.com/oss/javascript/langchain/supervisor